In [1]:
import numpy as np
import pandas as pd
import hanlp,joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
# 调用hanlp进行分词
hanlp.pretrained.tok.ALL
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
# 并行处理，将长评论进行分句同时处理
# 第一级管道分句，第二级管道分词和去停用词：
Hanlp = hanlp.pipeline().append(hanlp.utils.rules.split_sentence).append(tok).append(
    lambda sents: remove_stop_words(sents, '../stop.txt'))


#加载停用词词典
def load_file(filepath):
    with open(filepath, 'r', encoding="utf-8") as f:
        contents = f.readlines()
    result = []
    for content in contents:
        result.append(content.strip())
    return result


# 去除停用词并做特殊处理
def remove_stop_words(text, filepath):
    result = []
    str = ''
    dic = load_file(filepath)
    for k in text:
        for i in k:
            if i not in dic:
                result.append(i)
    # 对否定词“不、太”做特殊处理，把它与后面的词拼接
    while '不' in result:
        index = result.index('不')
        if index == len(result) - 1:
            break
        result[index:index + 2] = [''.join(result[index:index + 2])]
    str = " ".join(result)

    return str


def load_corpus(aaa):
    """
    df批处理
    """
    data = []
    for line in aaa:
        content = Hanlp(line)
        data.append(content)
    return data

In [3]:
df = pd.read_csv('../csv/test_label.csv')
df['content'] = load_corpus(df['content'])
df_train = df.sample(frac=0.6, replace=False, random_state=0, axis=0)
df_test = df[~df.index.isin(df_train.index)]
# vec=joblib.load('../bayes_train/v2/vec_model.m')
vec=joblib.load('../bayes_train/v1/vec_model.m')
x_train = vec.fit_transform(df_train['content'])
y_train = df_train['score']
x_test = vec.transform(df_test['content'])
y_test = df_test['score']
clf= joblib.load('../bayes_train/v1/clf_model6000.m')


In [4]:
bdt = AdaBoostClassifier(MultinomialNB(),algorithm='SAMME.R',n_estimators=200,learning_rate=0.8)
bdt.fit(x_train,y_train)
predictions = bdt.predict(x_test)
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))
print('准确率:', metrics.accuracy_score(y_test, predictions))
from pandas.core.frame import DataFrame
array = ['下雨，没感觉多好，尤其是玻璃栈道，没感觉就过去了，导游很热情，服务到位',
    '1.安排住宿酒店离市区都很远，环境也不足五星标准，行程团餐很难吃2.第四天下午去大理，没有提前告诉我高铁票号，小孩出行用的是户口薄，需要有票号才能取，我到了高铁站工作人员告诉我要票号，我问导游她竟然不知道，还要再去问公司，折腾了好久，另外15：55发车的高铁，13点就把我们送到高铁站，足足等了3小时3.住的酒店离昆明高铁南站只有10分钟路程，为什么让我们昆明站下再坐1小时车到酒店？浪费时间订了那么多次携程，这次的体验差到爆',
    '整个行程几乎都是在交通工具上，坐动车，坐船，坐大巴车，真正玩的时间就半天。第二天回武汉的动车票是晚上9点半的，回到武汉都晚上12点过了，太晚了。行程无任何亮点，都是5个小景点，连神龙架的招牌神农祭坛都不在行程中，来回武汉花了将近一天的时间，景区参观加起来不到3小时，有种被忽悠的感觉，别人问我都不好意思说去过神龙架还有不包伙食的，要交35元每人每餐，第一餐就是船上的食堂盒饭，菜很差，吃完之后盘子就放那种 ** * 桶里。晚上和第二天中午在木鱼镇一个食堂吃的，还过的去，住的神龙山庄酒店还可以，酒店早餐也可以。乘船有2层，一层大厅免费的，但靠窗的位置一桌100元，二楼一桌180元。同样的行程，同一个团的，在途牛网预订的就要便宜几十元。携程就是贵。',
    '旅游团按排的太差，导游讲解不尽责尽力，随便讲一下，不跟团讲解，自己在门口休息，不???导游客如何烧香拜佛，一个非常不称职的导游!',
    '时间安排成问题，各种等待，各种坑。早上五点起来说滑雪场排队。可等导游等了2个多小时。导游素质成问题。因为没有吃40 / 人的团餐。被安排到比别人差的酒店。气愤。',
    '非常差，这是我跟过最差的团', '额，说点啥子好捏，其它的不说，导游的在车上说的一些话反正一定伤到人的心了，，为什么给5星的，因为我也做过服务行业，我确实不忍心，在我眼里只有一星跟五星。',
    '行程安排紧凑，导游热情客气，家庭出游带父母小孩非常合适，能照顾到老人的感受和身体，酒店住的碧桂园，风景不错。张家界和天门山景色优美值得前来游玩',
    '旅行很愉快导游照顾很周到风景很美的一路很顺利导游安排的很妥当过程很快乐也很省事感谢导游的用心安排',
    '导游武阿哥，途中那个男的人很好，服务热情❤，从接团到散团我们的问题都比较多，因为第一次来湖南吖，想了解的也比较多，问题也是特别多????，大晚上给他打电话还耐心解答，十分感谢你的耐心????✌。本次出游很满意????????????']
df = DataFrame({'content': array})
data = vec.transform(df['content'])
pre = bdt.predict(data)
yy_pre = bdt.predict_proba(data)
print(pre,yy_pre)

              precision    recall  f1-score   support

           0       0.89      0.97      0.93        73
           1       0.95      0.80      0.87        45

    accuracy                           0.91       118
   macro avg       0.92      0.89      0.90       118
weighted avg       0.91      0.91      0.90       118

准确率: 0.9067796610169492
[0 0 0 0 0 0 0 0 0 0] [[0.50684742 0.49315258]
 [0.50844213 0.49155787]
 [0.50814259 0.49185741]
 [0.50840364 0.49159636]
 [0.50769266 0.49230734]
 [0.50840364 0.49159636]
 [0.50840364 0.49159636]
 [0.50840364 0.49159636]
 [0.50840364 0.49159636]
 [0.50840364 0.49159636]]


[0 0 0 0 0 0 0 0 0] [[0.50648782 0.49351218]
 [0.50625848 0.49374152]
 [0.5059064  0.4940936 ]
 [0.506473   0.493527  ]
 [0.506473   0.493527  ]
 [0.506473   0.493527  ]
 [0.506473   0.493527  ]
 [0.506473   0.493527  ]
 [0.506473   0.493527  ]]


In [6]:
# joblib.dump(bdt,'boost.m')
# joblib.dump(vec,'boost_vec.m')
# print('ok')

ok


In [25]:
for i in range(0,len(y_test.tolist())):
    print(y_test.tolist()[i],predictions[i])


0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
1 0
1 0
0 0
0 0
0 0
0 1
0 0
1 1
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
1 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
1 1
0 0
1 0
0 0
0 0
0 0
0 0
0 0
1 0
1 1
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
1 1
1 0
0 0
1 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
1 0
0 0
1 1
0 1
1 1
1 1
0 0
0 0
0 0
0 0
1 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 1
0 0
0 0
0 0
0 0
1 0
0 0
1 1
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
1 1
1 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
1 1
1 1
0 0
0 0
0 1


In [5]:
tf = pd.read_csv('../csv/test_label.csv')
tf['content']  = load_corpus(tf['content'])
tx_test = vec.transform(tf['content'])
ty_test = tf['score']
bddd  = joblib.load('boost.m')
tpre = bddd.predict(tx_test)
print(metrics.classification_report(ty_test, tpre))
print('准确率:', metrics.accuracy_score(ty_test, tpre))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       174
           1       0.98      0.92      0.95       121

    accuracy                           0.96       295
   macro avg       0.96      0.95      0.96       295
weighted avg       0.96      0.96      0.96       295

准确率: 0.9593220338983051


In [77]:
def load_text(aaa):
    """
    df批处理
    """
    # 并行处理，将长评论进行分句同时处理
    # 第一级管道分句，第二级管道分词和去停用词：
    data = []
    content = Hanlp(aaa)
    data.append(content)
    return data

content= "我们是提前预定的，1月9号就预定了，如果我早知道1月20号疫情会如此严重我绝对，绝对不会去预定这个行程。后来20号疫情严重化，本人又感冒，于是申请了退款，抱歉我们也是十分的不想退，携程客服打电话就一句话：退了，钱不退。我？？？于是我们迎难而上。***的操作来了，也就是景区，年初一，25号，因为***被迫关了，暂停接待，之前很多人都在建议关了，景区老总陈向宏发了微博，如图，别说我都感动的哭了，当然如果没有年初一的骚操作的话，我估计都会觉得大家都不容易。年初一，我们年三十提前一天的游客，就如同丧家犬一样，中午没有卖吃的了！！！对，景区卖吃的都关了，小景点也都关了。我们年初一就吃了一顿早饭，酒店自助。饿着肚子走的。如果我对退款不满意的是携程，是旅行社，那么景区这一波骚操作是人能干的事吗？游客还没走完，就关店了？想赚钱的时候，情怀就来了，说着如图所示的话，您03年记得别人无助的身影，为什么20年了，最后一天都不愿意服务前一天的游客呢？我们年三十入住的游客，迎难而上的游客，硬着头皮去的游客，已经抱着很好的心态去了，结果呢？是的，25号关就关，管24号游客是否能吃上饭，是否会有温暖。"

ttpre = bddd.predict(vec.transform(load_text(content))).tolist()

NameError: name 'bddd' is not defined

In [100]:
print(ttpre)


[1]
我们是提前预定的，1月9号就预定了，如果我早知道1月20号疫情会如此严重我绝对，绝对不会去预定这个行程。后来20号疫情严重化，本人又感冒，于是申请了退款，抱歉我们也是十分的不想退，携程客服打电话就一句话：退了，钱不退。我？？？于是我们迎难而上。***的操作来了，也就是景区，年初一，25号，因为***被迫关了，暂停接待，之前很多人都在建议关了，景区老总陈向宏发了微博，如图，别说我都感动的哭了，当然如果没有年初一的骚操作的话，我估计都会觉得大家都不容易。年初一，我们年三十提前一天的游客，就如同丧家犬一样，中午没有卖吃的了！！！对，景区卖吃的都关了，小景点也都关了。我们年初一就吃了一顿早饭，酒店自助。饿着肚子走的。如果我对退款不满意的是携程，是旅行社，那么景区这一波骚操作是人能干的事吗？游客还没走完，就关店了？想赚钱的时候，情怀就来了，说着如图所示的话，您03年记得别人无助的身影，为什么20年了，最后一天都不愿意服务前一天的游客呢？我们年三十入住的游客，迎难而上的游客，硬着头皮去的游客，已经抱着很好的心态去了，结果呢？是的，25号关就关，管24号游客是否能吃上饭，是否会有温暖。
